In [66]:
import os
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv, find_dotenv

# Carga las variables de entorno desde el archivo .env
_ = load_dotenv(find_dotenv())

In [67]:
# Usar parámetros específicos para evitar problemas de serialización
chat = ChatOllama(
    model='gemma3n:latest',
    temperature=0.7,
    # Agregar configuración para evitar problemas de serialización
    base_url="http://localhost:11434",
    
)

# Definimos una lista de mensajes para enviar al chatbot

In [68]:
from langchain_core.messages import HumanMessage

mensajes_para_el_chatbot =[HumanMessage(content='Mi Nombre es Manuel')]

##### Enviamos el mensaje al modelo

In [69]:
chat.invoke(mensajes_para_el_chatbot)

AIMessage(content='¡Hola Manuel! Un gusto conocerte. ¿En qué puedo ayudarte hoy? 😊 \n\n(Hello Manuel! Nice to meet you. How can I help you today?)\n', response_metadata={'model': 'gemma3n:latest', 'created_at': '2025-08-15T15:43:11.406958271Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17124493737, 'load_duration': 11783302423, 'prompt_eval_count': 13, 'prompt_eval_duration': 670382321, 'eval_count': 37, 'eval_duration': 4669248188, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)}, id='run-ae69de14-bfdc-45ed-97b1-20863f4607b1-0', usage_metadata={'input_tokens': 13, 'output_tokens': 37, 'total_tokens': 50})

preguntamos al modelo para saber si recuerda lo que le hemos preguntado

In [70]:
chat.invoke([HumanMessage(content='Como me llamo?')])

AIMessage(content='Soy un modelo de lenguaje grande, entrenado por Google. No tengo nombre propio. Puedes llamarme simplemente "el modelo de lenguaje" o "la IA". 😊\n', response_metadata={'model': 'gemma3n:latest', 'created_at': '2025-08-15T15:43:16.551730605Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5131585350, 'load_duration': 110446970, 'prompt_eval_count': 14, 'prompt_eval_duration': 583502397, 'eval_count': 35, 'eval_duration': 4437152496, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)}, id='run-556dfbec-ec68-42de-b97b-0cffe900d012-0', usage_metadata={'input_tokens': 14, 'output_tokens': 35, 'total_tokens': 49})

## Agreguemos memoria a nuestro Chatbot
* Usaremos el paquete ChatMessageHistory.
* Guardaremos la memoria del Chatbot en un diccionario de python llamado chatbotMemory.
* Definiremos la función get_session_history para crear un session_id para cada conversación.
* Usaremos el runnable incorporado RunnableWithMesageHistory.

In [71]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Diccionario para almacenar la memoria del chatbot por sesión
chatMemory = {}

# Función para obtener el historial de chat de una sesión
# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # Si no existe una entrada para este session_id, crea una nueva
    if session_id not in chatMemory:
        chatMemory[session_id] = ChatMessageHistory()
    return chatMemory[session_id]

# Crea un chatbot con historial de mensajes
chatbot_con_memoria = RunnableWithMessageHistory(
    chat, 
    get_session_history
)

In [72]:
session1= {'configurable':{'session_id':'001'}}

In [85]:
mensaje = [HumanMessage(content="Mi Nombre es Manuel")]

In [86]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
respuesta = chatbot_con_memoria.invoke(mensaje
    ,
    config=session1
)
chatMemory['001'].add_message(HumanMessage(content="Mi Nombre es Manuel"))

# Imprime el contenido de la respuesta del chatbot
respuesta.content

NotImplementedError in RootListenersTracer.on_llm_end callback: NotImplementedError('Trying to load an object that doesn\'t implement serialization: {\'lc\': 1, \'type\': \'not_implemented\', \'id\': [\'ollama\', \'_types\', \'Message\'], \'repr\': "Message(role=\'assistant\', content=\'\', thinking=None, images=None, tool_name=None, tool_calls=None)"}')


'¡Hola Manuel! Veo que estás confirmando tu nombre. ¡Bienvenido! 😊\n\n¿Hay algo en particular que te gustaría que hiciéramos? ¿Tienes alguna pregunta, necesitas ayuda con algo, o simplemente quieres conversar? \n\nEstoy aquí para ayudarte en lo que pueda.'

In [82]:
print(chatMemory['001'].messages)

[HumanMessage(content='Mi Nombre es Manuel'), AIMessage(content='¡Hola Manuel! Un gusto conocerte. ¿En qué puedo ayudarte hoy? 😊 \n\nDime, ¿tienes alguna pregunta, necesitas información sobre algo, o simplemente quieres charlar? Estoy aquí para lo que necesites.\n', response_metadata={'model': 'gemma3n:latest', 'created_at': '2025-08-15T15:44:58.839957607Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6198925953, 'load_duration': 125159703, 'prompt_eval_count': 13, 'prompt_eval_duration': 401563487, 'eval_count': 49, 'eval_duration': 5671787524, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)}, id='run-aa2a833f-bd46-4bfb-8094-ebbd0ad0ad1d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 49, 'total_tokens': 62}), HumanMessage(content='Mi Nombre es Manuel')]


In [ ]:
respuesta = chatbot_con_memoria.invoke(
    [HumanMessage(content='Como me llamo?')],
    config=session1)
respuesta.content


NotImplementedError in RootListenersTracer.on_llm_end callback: NotImplementedError('Trying to load an object that doesn\'t implement serialization: {\'lc\': 1, \'type\': \'not_implemented\', \'id\': [\'ollama\', \'_types\', \'Message\'], \'repr\': "Message(role=\'assistant\', content=\'\', thinking=None, images=None, tool_name=None, tool_calls=None)"}')


'¡Tu nombre es Manuel! Ya me lo dijiste al principio. 😊 \n\n¿Estás probando si estoy prestando atención? ¡Lo estoy! 😉\n'

In [84]:
print(chatMemory['001'].messages)

[HumanMessage(content='Mi Nombre es Manuel'), AIMessage(content='¡Hola Manuel! Un gusto conocerte. ¿En qué puedo ayudarte hoy? 😊 \n\nDime, ¿tienes alguna pregunta, necesitas información sobre algo, o simplemente quieres charlar? Estoy aquí para lo que necesites.\n', response_metadata={'model': 'gemma3n:latest', 'created_at': '2025-08-15T15:44:58.839957607Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6198925953, 'load_duration': 125159703, 'prompt_eval_count': 13, 'prompt_eval_duration': 401563487, 'eval_count': 49, 'eval_duration': 5671787524, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)}, id='run-aa2a833f-bd46-4bfb-8094-ebbd0ad0ad1d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 49, 'total_tokens': 62}), HumanMessage(content='Mi Nombre es Manuel')]
